In [25]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold
from sklearn.linear_model import LinearRegression,LogisticRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
import itertools as it
import time as time
from catboost import CatBoostRegressor
import xgboost as xgb
from sklearn.ensemble import StackingRegressor

In [29]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
nan_cols = test.isna().sum()
test1 = test.drop(nan_cols[nan_cols > 269].index.tolist(), axis = 1)
train1 = train.drop(nan_cols[nan_cols > 269].index.tolist(), axis = 1)

from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3)

# fit and transform the data
test_imputed = pd.DataFrame(imputer.fit_transform(test1), columns=test1.columns)
train_imputed = pd.DataFrame(imputer.fit_transform(train1), columns=train1.columns)

corr_matrix = train_imputed.corr().abs()

corr_y = train_imputed.corrwith(train_imputed['y'])
train_clean = train_imputed.drop(corr_y[corr_y.isna()].index.tolist(), axis = 1)
test_clean = test_imputed.drop(corr_y[corr_y.isna()].index.tolist(), axis = 1)

# identify predictors that have correlation of > 0.85
indices = np.where(np.triu(corr_matrix > 0.9, k=1))

# print the pairs of columns that have a correlation > 0.85
high_corr = []
count = 0
for idx in range(len(indices[0])):
    col1 = corr_matrix.columns[indices[0][idx]]
    col2 = corr_matrix.columns[indices[1][idx]]
    high_corr.append(col1)
    count+=1
    #print(f"{col1} and {col2}")
#print(count)

unique_vals = set(high_corr)
train_clean = train_clean.drop(unique_vals, axis = 1)

test_col = train_clean.drop('y', axis = 1)
test_clean = test_clean[test_col.columns]

In [30]:
corr_matrix = train_clean.corr().abs()

corr_y = train_clean.corrwith(train_clean['y'])

low_corr_cols = corr_y[corr_y < 0.03].index.tolist() # 0.01 was best so far 

train_clean = train_clean.drop(low_corr_cols, axis = 1)
test_col = train_clean.drop('y', axis = 1)
test_clean = test_clean[test_col.columns]

X = train_clean.drop('y', axis = 1)
y = train_clean.y

X_test = test_clean

In [10]:
X_test.shape

(4403, 59)

In [17]:
corr_y = train_clean.corrwith(train_clean['y'])
low_corr_cols = corr_y[corr_y < 0.03].index.tolist() # 0.01 was best so far 
len(low_corr_cols)

181

In [19]:
X['x102'] = train_imputed['x102']
X['x561'] = train_imputed['x561']
X['x355'] = train_imputed['x355']
X['x585'] = train_imputed['x585']

X_test['x102'] = test_imputed['x102']
X_test['x561'] = test_imputed['x561']
X_test['x355'] = test_imputed['x355']
X_test['x585'] = test_imputed['x585']

In [20]:
xgb_model = xgb.XGBRegressor(random_state=1,max_depth=6,n_estimators=500,learning_rate=0.01,reg_lambda=0.001)
xgb_model.fit(X,y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [21]:
catboost = CatBoostRegressor(subsample=0.5,
                          reg_lambda=10, 
                          num_leaves=31, 
                          n_estimators=500, 
                          max_depth=6, 
                          learning_rate=0.05)
catboost.fit(X,y)

0:	learn: 11.8941262	total: 63.1ms	remaining: 31.5s
1:	learn: 11.7700872	total: 72.1ms	remaining: 18s
2:	learn: 11.6595817	total: 80.6ms	remaining: 13.3s
3:	learn: 11.5628776	total: 90.1ms	remaining: 11.2s
4:	learn: 11.4610455	total: 108ms	remaining: 10.7s
5:	learn: 11.3677103	total: 120ms	remaining: 9.87s
6:	learn: 11.2768311	total: 141ms	remaining: 9.92s
7:	learn: 11.1968836	total: 166ms	remaining: 10.2s
8:	learn: 11.1202789	total: 180ms	remaining: 9.84s
9:	learn: 11.0398665	total: 193ms	remaining: 9.45s
10:	learn: 10.9649413	total: 203ms	remaining: 9.01s
11:	learn: 10.8832764	total: 214ms	remaining: 8.68s
12:	learn: 10.8031021	total: 221ms	remaining: 8.28s
13:	learn: 10.7321682	total: 231ms	remaining: 8.02s
14:	learn: 10.6794119	total: 243ms	remaining: 7.86s
15:	learn: 10.6107888	total: 253ms	remaining: 7.66s
16:	learn: 10.5576305	total: 265ms	remaining: 7.54s
17:	learn: 10.5020142	total: 278ms	remaining: 7.46s
18:	learn: 10.4427816	total: 286ms	remaining: 7.24s
19:	learn: 10.379749

166:	learn: 8.4264091	total: 1.93s	remaining: 3.84s
167:	learn: 8.4228567	total: 1.94s	remaining: 3.84s
168:	learn: 8.4224195	total: 1.96s	remaining: 3.83s
169:	learn: 8.4183746	total: 1.97s	remaining: 3.82s
170:	learn: 8.4162236	total: 1.98s	remaining: 3.81s
171:	learn: 8.4156512	total: 1.99s	remaining: 3.79s
172:	learn: 8.4123730	total: 2s	remaining: 3.77s
173:	learn: 8.4089749	total: 2.02s	remaining: 3.78s
174:	learn: 8.3967432	total: 2.03s	remaining: 3.77s
175:	learn: 8.3943125	total: 2.04s	remaining: 3.75s
176:	learn: 8.3895498	total: 2.05s	remaining: 3.75s
177:	learn: 8.3831707	total: 2.07s	remaining: 3.75s
178:	learn: 8.3763843	total: 2.09s	remaining: 3.75s
179:	learn: 8.3740995	total: 2.1s	remaining: 3.73s
180:	learn: 8.3735914	total: 2.12s	remaining: 3.73s
181:	learn: 8.3631224	total: 2.13s	remaining: 3.73s
182:	learn: 8.3587166	total: 2.15s	remaining: 3.73s
183:	learn: 8.3577958	total: 2.17s	remaining: 3.73s
184:	learn: 8.3529195	total: 2.19s	remaining: 3.72s
185:	learn: 8.34

325:	learn: 7.6807255	total: 4.57s	remaining: 2.44s
326:	learn: 7.6796175	total: 4.58s	remaining: 2.42s
327:	learn: 7.6770573	total: 4.59s	remaining: 2.41s
328:	learn: 7.6734255	total: 4.61s	remaining: 2.4s
329:	learn: 7.6709262	total: 4.63s	remaining: 2.39s
330:	learn: 7.6570388	total: 4.64s	remaining: 2.37s
331:	learn: 7.6533592	total: 4.66s	remaining: 2.36s
332:	learn: 7.6467402	total: 4.67s	remaining: 2.34s
333:	learn: 7.6447440	total: 4.68s	remaining: 2.33s
334:	learn: 7.6352471	total: 4.69s	remaining: 2.31s
335:	learn: 7.6331878	total: 4.7s	remaining: 2.29s
336:	learn: 7.6253582	total: 4.71s	remaining: 2.28s
337:	learn: 7.6181664	total: 4.72s	remaining: 2.26s
338:	learn: 7.6151341	total: 4.73s	remaining: 2.25s
339:	learn: 7.6121987	total: 4.74s	remaining: 2.23s
340:	learn: 7.6066309	total: 4.75s	remaining: 2.21s
341:	learn: 7.5998032	total: 4.76s	remaining: 2.2s
342:	learn: 7.5870370	total: 4.79s	remaining: 2.19s
343:	learn: 7.5867139	total: 4.8s	remaining: 2.18s
344:	learn: 7.57

487:	learn: 7.1023273	total: 6.5s	remaining: 160ms
488:	learn: 7.0972940	total: 6.54s	remaining: 147ms
489:	learn: 7.0918434	total: 6.56s	remaining: 134ms
490:	learn: 7.0894388	total: 6.57s	remaining: 121ms
491:	learn: 7.0791931	total: 6.6s	remaining: 107ms
492:	learn: 7.0771830	total: 6.63s	remaining: 94.2ms
493:	learn: 7.0749933	total: 6.66s	remaining: 80.8ms
494:	learn: 7.0733709	total: 6.67s	remaining: 67.4ms
495:	learn: 7.0669915	total: 6.68s	remaining: 53.9ms
496:	learn: 7.0654267	total: 6.7s	remaining: 40.4ms
497:	learn: 7.0636005	total: 6.72s	remaining: 27ms
498:	learn: 7.0593667	total: 6.73s	remaining: 13.5ms
499:	learn: 7.0563083	total: 6.74s	remaining: 0us


In [22]:
#Finding the lasso coefficients for increasing values of the tuning parameter
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV

alphas = 10**np.linspace(10,-2,100)*0.1
lasso = Lasso(max_iter = 10000)
coefs = []

for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(X,y)
    coefs.append(lasso.coef_)

alphas = 10**np.linspace(-1,-5,200)*0.5
lassocv = LassoCV(alphas = alphas, cv = 10, max_iter = 100000)
lassocv.fit(X,y)

#Optimal value of the tuning parameter - lamda
lassocv.alpha_

0.012472540676151583

In [23]:
lasso = Lasso(alpha = lassocv.alpha_)
lasso.fit(X,y)

Lasso(alpha=0.012472540676151583)

In [26]:
en = StackingRegressor(estimators = [('lasso', lasso),('catboost', catboost),('xgb', xgb_model)],
                     final_estimator=LinearRegression(),                                          
                    cv = KFold(n_splits = 5, shuffle = True, random_state=1))
en.fit(X,y)

0:	learn: 11.8941262	total: 10.7ms	remaining: 5.32s
1:	learn: 11.7700872	total: 18.1ms	remaining: 4.5s
2:	learn: 11.6595817	total: 36.9ms	remaining: 6.11s
3:	learn: 11.5628776	total: 44.1ms	remaining: 5.46s
4:	learn: 11.4610455	total: 51.2ms	remaining: 5.06s
5:	learn: 11.3677103	total: 58.2ms	remaining: 4.79s
6:	learn: 11.2768311	total: 68ms	remaining: 4.79s
7:	learn: 11.1968836	total: 77.1ms	remaining: 4.74s
8:	learn: 11.1202789	total: 90.7ms	remaining: 4.95s
9:	learn: 11.0398665	total: 99.3ms	remaining: 4.87s
10:	learn: 10.9649413	total: 106ms	remaining: 4.73s
11:	learn: 10.8832764	total: 113ms	remaining: 4.61s
12:	learn: 10.8031021	total: 120ms	remaining: 4.51s
13:	learn: 10.7321682	total: 133ms	remaining: 4.61s
14:	learn: 10.6794119	total: 143ms	remaining: 4.64s
15:	learn: 10.6107888	total: 151ms	remaining: 4.58s
16:	learn: 10.5576305	total: 159ms	remaining: 4.51s
17:	learn: 10.5020142	total: 166ms	remaining: 4.44s
18:	learn: 10.4427816	total: 186ms	remaining: 4.72s
19:	learn: 10.3

172:	learn: 8.4123730	total: 2.08s	remaining: 3.94s
173:	learn: 8.4089749	total: 2.09s	remaining: 3.92s
174:	learn: 8.3967432	total: 2.12s	remaining: 3.93s
175:	learn: 8.3943125	total: 2.12s	remaining: 3.91s
176:	learn: 8.3895498	total: 2.13s	remaining: 3.89s
177:	learn: 8.3831707	total: 2.14s	remaining: 3.87s
178:	learn: 8.3763843	total: 2.15s	remaining: 3.85s
179:	learn: 8.3740995	total: 2.16s	remaining: 3.85s
180:	learn: 8.3735914	total: 2.17s	remaining: 3.83s
181:	learn: 8.3631224	total: 2.18s	remaining: 3.81s
182:	learn: 8.3587166	total: 2.19s	remaining: 3.79s
183:	learn: 8.3577958	total: 2.2s	remaining: 3.77s
184:	learn: 8.3529195	total: 2.22s	remaining: 3.77s
185:	learn: 8.3454739	total: 2.23s	remaining: 3.76s
186:	learn: 8.3350222	total: 2.24s	remaining: 3.75s
187:	learn: 8.3233463	total: 2.25s	remaining: 3.73s
188:	learn: 8.3201031	total: 2.26s	remaining: 3.72s
189:	learn: 8.3159187	total: 2.27s	remaining: 3.71s
190:	learn: 8.3155571	total: 2.28s	remaining: 3.68s
191:	learn: 8

333:	learn: 7.6447440	total: 3.49s	remaining: 1.73s
334:	learn: 7.6352471	total: 3.53s	remaining: 1.74s
335:	learn: 7.6331878	total: 3.53s	remaining: 1.73s
336:	learn: 7.6253582	total: 3.54s	remaining: 1.71s
337:	learn: 7.6181664	total: 3.55s	remaining: 1.7s
338:	learn: 7.6151341	total: 3.57s	remaining: 1.7s
339:	learn: 7.6121987	total: 3.58s	remaining: 1.69s
340:	learn: 7.6066309	total: 3.6s	remaining: 1.68s
341:	learn: 7.5998032	total: 3.62s	remaining: 1.67s
342:	learn: 7.5870370	total: 3.63s	remaining: 1.66s
343:	learn: 7.5867139	total: 3.64s	remaining: 1.65s
344:	learn: 7.5792276	total: 3.66s	remaining: 1.64s
345:	learn: 7.5738605	total: 3.67s	remaining: 1.63s
346:	learn: 7.5703465	total: 3.67s	remaining: 1.62s
347:	learn: 7.5670468	total: 3.68s	remaining: 1.61s
348:	learn: 7.5650800	total: 3.69s	remaining: 1.59s
349:	learn: 7.5584811	total: 3.7s	remaining: 1.59s
350:	learn: 7.5565179	total: 3.71s	remaining: 1.58s
351:	learn: 7.5528322	total: 3.72s	remaining: 1.56s
352:	learn: 7.55

0:	learn: 11.6037918	total: 7.45ms	remaining: 3.72s
1:	learn: 11.4857361	total: 14.1ms	remaining: 3.51s
2:	learn: 11.3791731	total: 20.7ms	remaining: 3.42s
3:	learn: 11.2758278	total: 26.9ms	remaining: 3.34s
4:	learn: 11.1848308	total: 33.8ms	remaining: 3.34s
5:	learn: 11.0975757	total: 41.7ms	remaining: 3.43s
6:	learn: 11.0131879	total: 50.2ms	remaining: 3.54s
7:	learn: 10.9274396	total: 56.7ms	remaining: 3.48s
8:	learn: 10.8542545	total: 62.9ms	remaining: 3.43s
9:	learn: 10.7820404	total: 69.9ms	remaining: 3.42s
10:	learn: 10.7133590	total: 78.9ms	remaining: 3.51s
11:	learn: 10.6271339	total: 90.5ms	remaining: 3.68s
12:	learn: 10.5619606	total: 97.3ms	remaining: 3.64s
13:	learn: 10.5022075	total: 104ms	remaining: 3.59s
14:	learn: 10.4302903	total: 110ms	remaining: 3.55s
15:	learn: 10.3707333	total: 116ms	remaining: 3.51s
16:	learn: 10.3217637	total: 124ms	remaining: 3.53s
17:	learn: 10.2617905	total: 144ms	remaining: 3.85s
18:	learn: 10.2194200	total: 151ms	remaining: 3.83s
19:	learn

172:	learn: 8.1979241	total: 2.1s	remaining: 3.98s
173:	learn: 8.1920149	total: 2.12s	remaining: 3.97s
174:	learn: 8.1912773	total: 2.13s	remaining: 3.96s
175:	learn: 8.1818065	total: 2.15s	remaining: 3.95s
176:	learn: 8.1785153	total: 2.15s	remaining: 3.93s
177:	learn: 8.1774635	total: 2.16s	remaining: 3.91s
178:	learn: 8.1751190	total: 2.17s	remaining: 3.9s
179:	learn: 8.1716859	total: 2.19s	remaining: 3.89s
180:	learn: 8.1704648	total: 2.2s	remaining: 3.88s
181:	learn: 8.1626039	total: 2.21s	remaining: 3.86s
182:	learn: 8.1593598	total: 2.22s	remaining: 3.84s
183:	learn: 8.1537187	total: 2.23s	remaining: 3.82s
184:	learn: 8.1507038	total: 2.24s	remaining: 3.82s
185:	learn: 8.1498304	total: 2.25s	remaining: 3.81s
186:	learn: 8.1445597	total: 2.26s	remaining: 3.79s
187:	learn: 8.1357599	total: 2.27s	remaining: 3.77s
188:	learn: 8.1349789	total: 2.28s	remaining: 3.75s
189:	learn: 8.1306310	total: 2.29s	remaining: 3.74s
190:	learn: 8.1295156	total: 2.3s	remaining: 3.72s
191:	learn: 8.12

...

373:	learn: 7.4020485	total: 2.86s	remaining: 963ms
374:	learn: 7.3990312	total: 2.87s	remaining: 956ms
375:	learn: 7.3948876	total: 2.88s	remaining: 950ms
376:	learn: 7.3871068	total: 2.89s	remaining: 942ms
377:	learn: 7.3794114	total: 2.9s	remaining: 935ms
378:	learn: 7.3742309	total: 2.9s	remaining: 927ms
379:	learn: 7.3735470	total: 2.91s	remaining: 919ms
380:	learn: 7.3670348	total: 2.92s	remaining: 911ms
381:	learn: 7.3580256	total: 2.92s	remaining: 903ms
382:	learn: 7.3534977	total: 2.93s	remaining: 895ms
383:	learn: 7.3450148	total: 2.94s	remaining: 887ms
384:	learn: 7.3394259	total: 2.94s	remaining: 879ms
385:	learn: 7.3387672	total: 2.95s	remaining: 872ms
386:	learn: 7.3357785	total: 2.96s	remaining: 864ms
387:	learn: 7.3340145	total: 2.96s	remaining: 856ms
388:	learn: 7.3233345	total: 2.97s	remaining: 848ms
389:	learn: 7.3177839	total: 2.98s	remaining: 840ms
390:	learn: 7.3158363	total: 2.98s	remaining: 832ms
391:	learn: 7.3125404	total: 2.99s	remaining: 824ms
392:	learn: 7.

40:	learn: 9.6136790	total: 330ms	remaining: 3.7s
41:	learn: 9.5911153	total: 341ms	remaining: 3.72s
42:	learn: 9.5636212	total: 352ms	remaining: 3.74s
43:	learn: 9.5383952	total: 358ms	remaining: 3.71s
44:	learn: 9.5120883	total: 364ms	remaining: 3.68s
45:	learn: 9.4983522	total: 372ms	remaining: 3.67s
46:	learn: 9.4793059	total: 379ms	remaining: 3.66s
47:	learn: 9.4498721	total: 387ms	remaining: 3.64s
48:	learn: 9.4334047	total: 393ms	remaining: 3.61s
49:	learn: 9.4212682	total: 399ms	remaining: 3.59s
50:	learn: 9.4061598	total: 405ms	remaining: 3.56s
51:	learn: 9.3920659	total: 411ms	remaining: 3.54s
52:	learn: 9.3704500	total: 417ms	remaining: 3.52s
53:	learn: 9.3573048	total: 424ms	remaining: 3.5s
54:	learn: 9.3404727	total: 431ms	remaining: 3.49s
55:	learn: 9.3192715	total: 438ms	remaining: 3.47s
56:	learn: 9.2946744	total: 445ms	remaining: 3.46s
57:	learn: 9.2757420	total: 451ms	remaining: 3.44s
58:	learn: 9.2632817	total: 458ms	remaining: 3.43s
59:	learn: 9.2484634	total: 478ms

201:	learn: 8.1448406	total: 1.52s	remaining: 2.24s
202:	learn: 8.1428289	total: 1.54s	remaining: 2.25s
203:	learn: 8.1420970	total: 1.54s	remaining: 2.24s
204:	learn: 8.1394204	total: 1.56s	remaining: 2.24s
205:	learn: 8.1386528	total: 1.56s	remaining: 2.23s
206:	learn: 8.1357375	total: 1.57s	remaining: 2.22s
207:	learn: 8.1308507	total: 1.58s	remaining: 2.21s
208:	learn: 8.1282240	total: 1.58s	remaining: 2.2s
209:	learn: 8.1222785	total: 1.59s	remaining: 2.19s
210:	learn: 8.1183755	total: 1.6s	remaining: 2.19s
211:	learn: 8.1157072	total: 1.61s	remaining: 2.18s
212:	learn: 8.1133215	total: 1.61s	remaining: 2.17s
213:	learn: 8.1110053	total: 1.62s	remaining: 2.16s
214:	learn: 8.1079207	total: 1.63s	remaining: 2.15s
215:	learn: 8.1038158	total: 1.63s	remaining: 2.15s
216:	learn: 8.1006128	total: 1.64s	remaining: 2.14s
217:	learn: 8.0872482	total: 1.65s	remaining: 2.13s
218:	learn: 8.0809347	total: 1.66s	remaining: 2.12s
219:	learn: 8.0768871	total: 1.66s	remaining: 2.12s
220:	learn: 8.

362:	learn: 7.3563452	total: 2.71s	remaining: 1.02s
363:	learn: 7.3551316	total: 2.73s	remaining: 1.02s
364:	learn: 7.3507421	total: 2.73s	remaining: 1.01s
365:	learn: 7.3445648	total: 2.74s	remaining: 1s
366:	learn: 7.3433830	total: 2.75s	remaining: 996ms
367:	learn: 7.3425474	total: 2.75s	remaining: 988ms
368:	learn: 7.3352923	total: 2.76s	remaining: 980ms
369:	learn: 7.3254527	total: 2.77s	remaining: 974ms
370:	learn: 7.3239980	total: 2.78s	remaining: 966ms
371:	learn: 7.3175193	total: 2.79s	remaining: 958ms
372:	learn: 7.3144884	total: 2.79s	remaining: 951ms
373:	learn: 7.3121483	total: 2.8s	remaining: 943ms
374:	learn: 7.3052459	total: 2.8s	remaining: 935ms
375:	learn: 7.2988155	total: 2.81s	remaining: 927ms
376:	learn: 7.2951847	total: 2.82s	remaining: 921ms
377:	learn: 7.2831327	total: 2.83s	remaining: 913ms
378:	learn: 7.2793340	total: 2.83s	remaining: 905ms
379:	learn: 7.2778524	total: 2.84s	remaining: 897ms
380:	learn: 7.2683628	total: 2.85s	remaining: 889ms
381:	learn: 7.261

StackingRegressor(cv=KFold(n_splits=5, random_state=1, shuffle=True),
                  estimators=[('lasso', Lasso(alpha=0.012472540676151583)),
                              ('catboost',
                               <catboost.core.CatBoostRegressor object at 0x7fe030643160>),
                              ('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,...
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.01, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=6,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=500, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=1, ...))],
                  final_estimator=LinearRegression())

In [28]:
pred = pd.DataFrame()
pred['id'] = test['id']
pred['y'] = en.predict(X_test)
pred = pred.set_index(['id'])
pred.to_csv('Witarsa_Ashley_regression.csv')